In [1]:
import sys

sys.path.append("../")

from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from scipy.stats import entropy, kurtosis
from config.transformers import *
from config.models import *
from config.loss_functions import *
from config.validation import *
from config.regressors import *
from config.dann import *
import numpy as np
import pyriemann
import pywt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [2]:
PATH = r'C:\Users\gianm\Documents\Uni\Big Data\F422\project\data\\'
DATASET = 'freemoves' # change this to guided/freemoves if needed

X = np.load(PATH + f'{DATASET}/{DATASET}_dataset_X.npy')
Y = np.load(PATH + f'{DATASET}/{DATASET}_dataset_Y.npy')

tw_extractor = TimeWindowTransformer(size = 500, step = 50)
label_extractor = LabelWindowExtractor(size = 500, step = 50)

X_windows = tw_extractor.transform(X)
Y_labels = label_extractor.transform(Y)

In [3]:
rmse_scores = cross_validate_dann(X_windows, Y_labels, tensor_dataset=DANNWindowDataset)


=== Fold 1 | Train on [1, 2, 3, 4], Validate on 0 ===
Epoch 01 | Train Loss: 659.3960 | Val RMSE: 24.3763 | Dom Train Acc: 46.08%
           λ = 0.100
Epoch 02 | Train Loss: 600.7444 | Val RMSE: 22.9183 | Dom Train Acc: 46.62%
           λ = 0.200
Epoch 03 | Train Loss: 536.4845 | Val RMSE: 22.4063 | Dom Train Acc: 50.02%
           λ = 0.300
Epoch 04 | Train Loss: 470.5566 | Val RMSE: 20.5542 | Dom Train Acc: 49.53%
           λ = 0.400
Epoch 05 | Train Loss: 403.7856 | Val RMSE: 19.3817 | Dom Train Acc: 51.23%
           λ = 0.500


KeyboardInterrupt: 

In [3]:
def RMSE(y_pred, y_val):
    return (np.sum((y_val - y_pred) ** 2) / np.prod(y_val.shape)) ** 0.5

def NMSE(y_pred, y_val):
    num = np.sum((y_val - y_pred) ** 2)
    den = np.sum((y_val - np.mean(y_val, axis=0)) ** 2)
    return num / den

class DANNTrainer2:
    def __init__(
        self,
        model: nn.Module,
        X: np.ndarray,
        Y: np.ndarray,
        train_sessions: list,
        val_session: int,
        lambda_grl: float = 0.1,
        gamma_entropy=0.5,
        batch_size: int = 512,
        max_epochs: int = 50,
        patience: int = 10,
        learning_rate: float = 1e-3,
        device: str = "cuda" if torch.cuda.is_available() else "cpu",
        tensor_dataset=None
    ):
        self.device = device
        self.model = model.to(self.device)
        self.lambda_grl = lambda_grl
        self.gamma_entropy = gamma_entropy
        self.max_epochs = max_epochs
        self.patience = patience
        self.batch_size = batch_size
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=learning_rate)
        self.reg_loss = nn.MSELoss()
        self.dom_loss = nn.CrossEntropyLoss()
        self.train_loader = None  # to be assigned externally

    def train(self, validate=True):
        best_val_rmse = float("inf")
        best_weights = None
        patience_counter = 0

        def compute_lambda(epoch, max_lambda=1.0, warmup_epochs=10):
            return min(max_lambda, epoch / warmup_epochs)

        for epoch in range(1, self.max_epochs + 1):
            self.model.train()
            train_y_true, train_y_pred = [], []
            epoch_losses = []

            lambda_grl = compute_lambda(epoch, max_lambda=1.0, warmup_epochs=10)

            for x, y, sid in self.train_loader:
                x, y, sid = x.to(self.device), y.to(self.device), sid.to(self.device)
                y_pred, dom_pred = self.model(x, lambda_grl=lambda_grl)

                loss = (
                    self.reg_loss(y_pred, y) +
                    lambda_grl * self.dom_loss(dom_pred, sid)
                )
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                epoch_losses.append(loss.item())

                train_y_true.append(y.detach().cpu().numpy())
                train_y_pred.append(y_pred.detach().cpu().numpy())

            train_y_true = np.vstack(train_y_true)
            train_y_pred = np.vstack(train_y_pred)
            train_rmse = RMSE(train_y_pred, train_y_true)
            train_nmse = NMSE(train_y_pred, train_y_true)

            if validate:
                val_rmse = self.evaluate()
                domain_train_acc = self.evaluate_domain_on_train()
                print(f"Epoch {epoch:02d} | Train RMSE: {train_rmse:.4f} | Train NMSE: {train_nmse:.4f} | Val RMSE: {val_rmse:.4f} | Dom Train Acc: {domain_train_acc:.2%}")
                print(f"           λ = {lambda_grl:.3f}")

                if val_rmse < best_val_rmse:
                    best_val_rmse = val_rmse
                    best_weights = self.model.state_dict()
                    patience_counter = 0
                else:
                    patience_counter += 1
                    if patience_counter >= self.patience:
                        print("Early stopping triggered.")
                        break
            else:
                print(f"Epoch {epoch:02d} | Train RMSE: {train_rmse:.4f} | Train NMSE: {train_nmse:.4f}")
                print(f"           λ = {lambda_grl:.3f}")

        if validate and best_weights is not None:
            self.model.load_state_dict(best_weights)
        return best_val_rmse if validate else self.model

    def predict(self, data_loader):
        self.model.eval()
        all_preds = []
        with torch.no_grad():
            for x, *_ in data_loader:
                x = x.to(self.device)
                preds = self.model(x)[0]  # get y_pred only
                all_preds.append(preds.cpu().numpy())
        return np.vstack(all_preds)


In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset
from config.models import DANNTrainer

# --- Load full training set ---
PATH = r'C:\Users\gianm\Documents\Uni\Big Data\F422\project\data\\'
DATASET = 'freemoves'

X = np.load(PATH + f'{DATASET}/{DATASET}_dataset_X.npy')
Y = np.load(PATH + f'{DATASET}/{DATASET}_dataset_Y.npy')

step_prediction = 50
tw_extractor = TimeWindowTransformer(size=500, step=step_prediction)
label_extractor = LabelWindowExtractor(size=500, step=step_prediction)

# X_transformed = EMGPreprocessor().fit_transform(X)

X_windows = tw_extractor.transform(X)
Y_labels = label_extractor.transform(Y)

# X_all = X_windows.reshape(-1, 8, 500)
# Y_all = Y_labels.reshape(-1, 51)

session_ids = np.concatenate([
    np.full(X_windows[i].shape[0], i) for i in range(len(X_windows))
])

class DANNWindowDataset2(Dataset):
    def __init__(self, X, Y, session_ids):
        self.X = X.reshape(-1, *X.shape[2:])  # (N, 8, 500)
        self.Y = Y.reshape(-1, Y.shape[-1])   # (N, 51)
        self.session_ids = torch.tensor(session_ids, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (
            torch.tensor(self.X[idx], dtype=torch.float32),
            torch.tensor(self.Y[idx], dtype=torch.float32),
            self.session_ids[idx],
        )


train_dataset = DANNWindowDataset2(X_windows, Y_labels, session_ids)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

dummy_X = np.empty((1, 1, 8, 500), dtype=np.float32)
dummy_Y = np.empty((1, 1, 51), dtype=np.float32)
dummy_ids = np.zeros((1, 1), dtype=int)


# --- Define model ---
model = DANNModel(lambda_grl=1.0, num_domains=5, output_dim=51)

trainer = DANNTrainer2(
    model=model,
    X=dummy_X,
    Y=dummy_Y,
    train_sessions=[0],
    val_session=0,
    lambda_grl=1,
    gamma_entropy=0.4,
    batch_size=128,
    max_epochs=50,
    patience=10,
    learning_rate=1e-3,
    device='cuda',
    tensor_dataset=DANNWindowDataset2
)


# Inject trainer internals
trainer.train_loader = train_loader
trainer.val_loader = None
trainer.reg_loss = torch.nn.MSELoss()
trainer.dom_loss = torch.nn.CrossEntropyLoss()

# --- Train on full data ---
trainer.train(validate=False)

# --- Load test set ---
X_test = np.load(PATH + f"{DATASET}/{DATASET}_testset_X.npy")
X_test_windows = tw_extractor.transform(X_test).reshape(-1, 8, 500)
test_loader = DataLoader(
    TensorDataset(torch.tensor(X_test_windows, dtype=torch.float32)),
    batch_size=128, shuffle=False
)

# --- Predict and save ---
predictions = trainer.predict(test_loader)
np.save('dann_filterAct_epoch_50_steps_' + str(step_prediction) + '_rmse_6-8.npy', predictions)

Epoch 01 | Train RMSE: 23.1886 | Train NMSE: 3.3634
           λ = 0.100
Epoch 02 | Train RMSE: 16.4147 | Train NMSE: 1.6854
           λ = 0.200
Epoch 03 | Train RMSE: 11.9771 | Train NMSE: 0.8973
           λ = 0.300
Epoch 04 | Train RMSE: 10.4710 | Train NMSE: 0.6858
           λ = 0.400
Epoch 05 | Train RMSE: 9.9555 | Train NMSE: 0.6199
           λ = 0.500
Epoch 06 | Train RMSE: 9.7235 | Train NMSE: 0.5914
           λ = 0.600
Epoch 07 | Train RMSE: 9.5032 | Train NMSE: 0.5649
           λ = 0.700
Epoch 08 | Train RMSE: 9.3601 | Train NMSE: 0.5480
           λ = 0.800
Epoch 09 | Train RMSE: 9.2284 | Train NMSE: 0.5327
           λ = 0.900
Epoch 10 | Train RMSE: 9.1559 | Train NMSE: 0.5244
           λ = 1.000
Epoch 11 | Train RMSE: 9.0263 | Train NMSE: 0.5096
           λ = 1.000
Epoch 12 | Train RMSE: 8.9721 | Train NMSE: 0.5035
           λ = 1.000
Epoch 13 | Train RMSE: 8.8688 | Train NMSE: 0.4920
           λ = 1.000
Epoch 14 | Train RMSE: 8.7768 | Train NMSE: 0.4818
         

In [3]:
class DeltaFeatureTransformer(BaseEstimator, TransformerMixin):
    """
    Computes raw, delta, and delta^2 along time dimension of EMG windows,
    preserving session structure.

    Input shape:  (n_sessions, n_windows, n_channels, time)
    Output shape: (n_sessions, n_windows, n_channels * 3, time)
    """
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # X.shape = (n_sessions, n_windows, n_channels, time)
        raw = X  # shape: (S, W, C, T)
        diff = np.diff(X, axis=-1, prepend=X[..., :1])  # Δ
        diff2 = np.diff(diff, axis=-1, prepend=diff[..., :1])  # Δ²

        # Stack along a new "temporal feature" axis: raw, Δ, Δ²
        # Result: (S, W, C, T, 3)
        stacked = np.stack([raw, diff, diff2], axis=-1)

        # Rearrange to: (S, W, C * 3, T)
        S, W, C, T, F = stacked.shape
        output = stacked.transpose(0, 1, 2, 4, 3).reshape(S, W, C * F, T)

        return output

In [4]:
X = np.load(PATH + f'{DATASET}/{DATASET}_dataset_X.npy')
Y = np.load(PATH + f'{DATASET}/{DATASET}_dataset_Y.npy')

step_prediction = 50
tw_extractor = TimeWindowTransformer(size=500, step=step_prediction)
label_extractor = LabelWindowExtractor(size=500, step=step_prediction)

# X_transformed = EMGPreprocessor().fit_transform(X)

X_windows = tw_extractor.transform(X)
Y_labels = label_extractor.transform(Y)
X_transformed = DeltaFeatureTransformer().fit_transform(X_windows)
session_ids = np.repeat(np.arange(5), X.shape[1])

In [5]:
def cross_validate_dann(
    X,
    Y,
    session_ids,
    tensor_dataset,
    model_class,
    lambda_grl=0.3,
    max_epochs=50,
    patience=20,
    batch_size=512,
):
    import numpy as np

    n_sessions = X.shape[0]
    rmse_scores = []

    # Flatten X and Y *once*
    n_sessions, n_windows = X.shape[:2]
    X_flat = X.reshape(n_sessions * n_windows, *X.shape[2:])  # (S×W, C, T)
    Y_flat = Y.reshape(n_sessions * n_windows, Y.shape[-1])   # (S×W, D)

    # Rebuild session_ids if needed
    session_ids_flat = np.repeat(np.arange(n_sessions), n_windows)

    for val_session in range(n_sessions):
        train_sessions = [s for s in range(n_sessions) if s != val_session]
        print(f"\n=== Fold {val_session + 1} | Train on {train_sessions}, Validate on {val_session} ===")

        model = model_class() 

        trainer = DANNTrainer(
            model=model,
            X=None,
            Y=None,
            train_sessions=[],
            val_session=None,
            lambda_grl=lambda_grl,
            max_epochs=max_epochs,
            patience=patience,
            batch_size=batch_size,
            tensor_dataset=tensor_dataset
        )

        # Create correct boolean masks
        train_mask = session_ids_flat != val_session

        # Subset correctly
        X_train = X_flat[train_mask]
        Y_train = Y_flat[train_mask]
        session_ids_train = session_ids_flat[train_mask]

        train_dataset = tensor_dataset(X_train, Y_train, session_ids_train)
        trainer.train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
        val_mask = session_ids_flat == val_session

        X_val = X_flat[val_mask]
        Y_val = Y_flat[val_mask]
        session_ids_val = session_ids_flat[val_mask]
        
        val_dataset = tensor_dataset(X_val, Y_val, session_ids_val)
        trainer.val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

        rmse = trainer.train()
        rmse_scores.append(rmse)

    mean_rmse = np.mean(rmse_scores)
    std_rmse = np.std(rmse_scores)

    print(f"\n=== Cross-validated RMSE: {mean_rmse:.4f} ± {std_rmse:.4f} ===")
    return rmse_scores

In [6]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def build_model():
    model = TemporalDANNModel(lambda_grl=0.3, num_domains=5, output_dim=51)
    return model.to(device)


model = build_model() 

rmse_scores = cross_validate_dann(
    X_transformed,
    Y_labels,
    session_ids=session_ids,
    tensor_dataset=DANNWindowTensor,
    model_class=build_model,
    lambda_grl=0.3,
    max_epochs=50,
    patience=20,
    batch_size=2048
)



=== Fold 1 | Train on [1, 2, 3, 4], Validate on 0 ===
Epoch 01 | Train RMSE: 26.0696 | Train NMSE: 4.0955 | Val RMSE: 24.8626 | Dom Train Acc: 34.57%
           λ = 0.100


KeyboardInterrupt: 